In [1]:
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from glob import glob

In [2]:
working_path="/home/jtu9/lung/output/"
file_list=glob(working_path+"images_*.npy")

In [3]:
for img_file in file_list:
    # I ran into an error when using Kmean on np.float16, so I'm using np.float64 here
    imgs_to_process = np.load(img_file).astype(np.float64) 
    print ("on image", img_file)
    for i in range(len(imgs_to_process)):
        img = imgs_to_process[i]
        #Standardize the pixel values
        mean = np.mean(img)
        std = np.std(img)
        img = img-mean
        img = img/std
        # Find the average pixel value near the lungs
        # to renormalize washed out images
        middle = img[100:400,100:400] 
        mean = np.mean(middle)  
        max = np.max(img)
        min = np.min(img)
        # To improve threshold finding, I'm moving the 
        # underflow and overflow on the pixel spectrum
        img[img==max]=mean
        img[img==min]=mean
        #
        # Using Kmeans to separate foreground (radio-opaque tissue)
        # and background (radio transparent tissue ie lungs)
        # Doing this only on the center of the image to avoid 
        # the non-tissue parts of the image as much as possible
        #
        kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
        centers = sorted(kmeans.cluster_centers_.flatten())
        threshold = np.mean(centers)
        thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image
        #
        # I found an initial erosion helful for removing graininess from some of the regions
        # and then large dialation is used to make the lung region 
        # engulf the vessels and incursions into the lung cavity by 
        # radio opaque tissue
        #
        eroded = morphology.erosion(thresh_img,np.ones([4,4]))
        dilation = morphology.dilation(eroded,np.ones([10,10]))
        #
        #  Label each region and obtain the region properties
        #  The background region is removed by removing regions 
        #  with a bbox that is to large in either dimnsion
        #  Also, the lungs are generally far away from the top 
        #  and bottom of the image, so any regions that are too
        #  close to the top and bottom are removed
        #  This does not produce a perfect segmentation of the lungs
        #  from the image, but it is surprisingly good considering its
        #  simplicity. 
        #
        labels = measure.label(dilation)
        label_vals = np.unique(labels)
        regions = measure.regionprops(labels)
        good_labels = []
        for prop in regions:
            B = prop.bbox
            if B[2]-B[0]<475 and B[3]-B[1]<475 and B[0]>40 and B[2]<472:
                good_labels.append(prop.label)
        mask = np.ndarray([512,512],dtype=np.int8)
        mask[:] = 0
        #
        #  The mask here is the mask for the lungs--not the nodes
        #  After just the lungs are left, we do another large dilation
        #  in order to fill in and out the lung mask 
        #
        for N in good_labels:
            mask = mask + np.where(labels==N,1,0)
        mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
        imgs_to_process[i] = mask
    np.save(img_file.replace("images","lungmask"),imgs_to_process)
    

on image /home/jtu9/lung/output/images_0725_0347.npy
on image /home/jtu9/lung/output/images_0248_0174.npy
on image /home/jtu9/lung/output/images_0844_0085.npy
on image /home/jtu9/lung/output/images_0850_0611.npy
on image /home/jtu9/lung/output/images_0081_0094.npy
on image /home/jtu9/lung/output/images_0562_0355.npy
on image /home/jtu9/lung/output/images_0616_0317.npy
on image /home/jtu9/lung/output/images_0426_0462.npy
on image /home/jtu9/lung/output/images_0321_1116.npy
on image /home/jtu9/lung/output/images_0327_0939.npy
on image /home/jtu9/lung/output/images_0795_0261.npy
on image /home/jtu9/lung/output/images_0740_0163.npy
on image /home/jtu9/lung/output/images_0696_0582.npy
on image /home/jtu9/lung/output/images_0770_0989.npy
on image /home/jtu9/lung/output/images_0392_0885.npy
on image /home/jtu9/lung/output/images_0479_0333.npy
on image /home/jtu9/lung/output/images_0585_0309.npy
on image /home/jtu9/lung/output/images_0489_0912.npy
on image /home/jtu9/lung/output/images_0149_10

on image /home/jtu9/lung/output/images_0287_0897.npy
on image /home/jtu9/lung/output/images_0374_0640.npy
on image /home/jtu9/lung/output/images_0102_0718.npy
on image /home/jtu9/lung/output/images_0291_0435.npy
on image /home/jtu9/lung/output/images_0421_0123.npy
on image /home/jtu9/lung/output/images_0833_0698.npy
on image /home/jtu9/lung/output/images_0572_1091.npy
on image /home/jtu9/lung/output/images_0023_0075.npy
on image /home/jtu9/lung/output/images_0208_0495.npy
on image /home/jtu9/lung/output/images_0064_0395.npy
on image /home/jtu9/lung/output/images_0423_0973.npy
on image /home/jtu9/lung/output/images_0535_0450.npy
on image /home/jtu9/lung/output/images_0061_0745.npy
on image /home/jtu9/lung/output/images_0566_0926.npy
on image /home/jtu9/lung/output/images_0447_0196.npy
on image /home/jtu9/lung/output/images_0521_0490.npy
on image /home/jtu9/lung/output/images_0704_0561.npy
on image /home/jtu9/lung/output/images_0294_0335.npy
on image /home/jtu9/lung/output/images_0581_04

on image /home/jtu9/lung/output/images_0407_0889.npy
on image /home/jtu9/lung/output/images_0302_0384.npy
on image /home/jtu9/lung/output/images_0490_0992.npy
on image /home/jtu9/lung/output/images_0431_0236.npy
on image /home/jtu9/lung/output/images_0133_1122.npy
on image /home/jtu9/lung/output/images_0481_0708.npy
on image /home/jtu9/lung/output/images_0473_0905.npy
on image /home/jtu9/lung/output/images_0127_0802.npy
on image /home/jtu9/lung/output/images_0801_0602.npy
on image /home/jtu9/lung/output/images_0088_0787.npy
on image /home/jtu9/lung/output/images_0008_0696.npy
on image /home/jtu9/lung/output/images_0734_0586.npy
on image /home/jtu9/lung/output/images_0302_0383.npy
on image /home/jtu9/lung/output/images_0786_1052.npy
on image /home/jtu9/lung/output/images_0283_0072.npy
on image /home/jtu9/lung/output/images_0422_0256.npy
on image /home/jtu9/lung/output/images_0026_0868.npy
on image /home/jtu9/lung/output/images_0839_0533.npy
on image /home/jtu9/lung/output/images_0672_05

on image /home/jtu9/lung/output/images_0794_0895.npy
on image /home/jtu9/lung/output/images_0692_0031.npy
on image /home/jtu9/lung/output/images_0010_0372.npy
on image /home/jtu9/lung/output/images_0427_0962.npy
on image /home/jtu9/lung/output/images_0554_0419.npy
on image /home/jtu9/lung/output/images_0562_0354.npy
on image /home/jtu9/lung/output/images_0460_0122.npy
on image /home/jtu9/lung/output/images_0459_0968.npy
on image /home/jtu9/lung/output/images_0507_0847.npy
on image /home/jtu9/lung/output/images_0876_0753.npy
on image /home/jtu9/lung/output/images_0165_1089.npy
on image /home/jtu9/lung/output/images_0312_0113.npy
on image /home/jtu9/lung/output/images_0696_0581.npy
on image /home/jtu9/lung/output/images_0653_0494.npy
on image /home/jtu9/lung/output/images_0106_1025.npy
on image /home/jtu9/lung/output/images_0039_1144.npy
on image /home/jtu9/lung/output/images_0640_0479.npy
on image /home/jtu9/lung/output/images_0189_0180.npy
on image /home/jtu9/lung/output/images_0434_10

on image /home/jtu9/lung/output/images_0597_0131.npy
on image /home/jtu9/lung/output/images_0483_0311.npy
on image /home/jtu9/lung/output/images_0260_0880.npy
on image /home/jtu9/lung/output/images_0651_0944.npy
on image /home/jtu9/lung/output/images_0578_0666.npy
on image /home/jtu9/lung/output/images_0774_0899.npy
on image /home/jtu9/lung/output/images_0334_0155.npy
on image /home/jtu9/lung/output/images_0038_0068.npy
on image /home/jtu9/lung/output/images_0278_1076.npy
on image /home/jtu9/lung/output/images_0239_0791.npy
on image /home/jtu9/lung/output/images_0307_1169.npy
on image /home/jtu9/lung/output/images_0295_0882.npy
on image /home/jtu9/lung/output/images_0302_0386.npy
on image /home/jtu9/lung/output/images_0560_0474.npy
on image /home/jtu9/lung/output/images_0408_0517.npy
on image /home/jtu9/lung/output/images_0576_1158.npy
on image /home/jtu9/lung/output/images_0307_1166.npy
on image /home/jtu9/lung/output/images_0329_0059.npy
on image /home/jtu9/lung/output/images_0349_09

on image /home/jtu9/lung/output/images_0685_1090.npy
on image /home/jtu9/lung/output/images_0409_0852.npy
on image /home/jtu9/lung/output/images_0476_0957.npy
on image /home/jtu9/lung/output/images_0022_0339.npy
on image /home/jtu9/lung/output/images_0812_0601.npy
on image /home/jtu9/lung/output/images_0632_0821.npy
on image /home/jtu9/lung/output/images_0691_1001.npy
on image /home/jtu9/lung/output/images_0568_0644.npy
on image /home/jtu9/lung/output/images_0179_0716.npy
on image /home/jtu9/lung/output/images_0440_1057.npy
on image /home/jtu9/lung/output/images_0157_0293.npy
on image /home/jtu9/lung/output/images_0301_0200.npy
on image /home/jtu9/lung/output/images_0347_0959.npy
on image /home/jtu9/lung/output/images_0436_0406.npy
on image /home/jtu9/lung/output/images_0426_0465.npy
on image /home/jtu9/lung/output/images_0131_0672.npy
on image /home/jtu9/lung/output/images_0394_0576.npy
on image /home/jtu9/lung/output/images_0887_0472.npy
on image /home/jtu9/lung/output/images_0499_04

on image /home/jtu9/lung/output/images_0725_0349.npy
on image /home/jtu9/lung/output/images_0843_1065.npy
on image /home/jtu9/lung/output/images_0778_0030.npy
on image /home/jtu9/lung/output/images_0517_0867.npy
on image /home/jtu9/lung/output/images_0157_0290.npy
on image /home/jtu9/lung/output/images_0329_0058.npy
on image /home/jtu9/lung/output/images_0802_0476.npy
on image /home/jtu9/lung/output/images_0814_0389.npy
on image /home/jtu9/lung/output/images_0394_0569.npy
on image /home/jtu9/lung/output/images_0524_0614.npy
on image /home/jtu9/lung/output/images_0745_0877.npy
on image /home/jtu9/lung/output/images_0877_0553.npy
on image /home/jtu9/lung/output/images_0425_0820.npy
on image /home/jtu9/lung/output/images_0606_0626.npy
on image /home/jtu9/lung/output/images_0282_0191.npy
on image /home/jtu9/lung/output/images_0446_0431.npy
on image /home/jtu9/lung/output/images_0704_0562.npy
on image /home/jtu9/lung/output/images_0176_0555.npy
on image /home/jtu9/lung/output/images_0311_01

on image /home/jtu9/lung/output/images_0648_0356.npy
on image /home/jtu9/lung/output/images_0177_0965.npy
on image /home/jtu9/lung/output/images_0462_0226.npy
on image /home/jtu9/lung/output/images_0800_0598.npy
on image /home/jtu9/lung/output/images_0355_0794.npy
on image /home/jtu9/lung/output/images_0812_0600.npy
on image /home/jtu9/lung/output/images_0188_0769.npy
on image /home/jtu9/lung/output/images_0474_0150.npy
on image /home/jtu9/lung/output/images_0244_0946.npy
on image /home/jtu9/lung/output/images_0461_0189.npy
on image /home/jtu9/lung/output/images_0817_0446.npy
on image /home/jtu9/lung/output/images_0426_0463.npy
on image /home/jtu9/lung/output/images_0323_1110.npy
on image /home/jtu9/lung/output/images_0626_0305.npy
on image /home/jtu9/lung/output/images_0557_0003.npy
on image /home/jtu9/lung/output/images_0574_0398.npy
on image /home/jtu9/lung/output/images_0308_0168.npy
on image /home/jtu9/lung/output/images_0135_0023.npy
on image /home/jtu9/lung/output/images_0258_01

In [5]:

#
#    Here we're applying the masks and cropping and resizing the image
#

file_list=glob(working_path+"lungmask_*.npy")
out_images = []      #final set of images
out_nodemasks = []   #final set of nodemasks
for fname in file_list:
    print ("working on file ", fname)
    imgs_to_process = np.load(fname.replace("lungmask","images"))
    masks = np.load(fname)
    node_masks = np.load(fname.replace("lungmask","masks"))
    for i in range(len(imgs_to_process)):
        mask = masks[i]
        node_mask = node_masks[i]
        img = imgs_to_process[i]
        new_size = [512,512]   # we're scaling back up to the original size of the image
        img= mask*img          # apply lung mask
        #
        # renormalizing the masked image (in the mask region)
        #
        new_mean = np.mean(img[mask>0])  
        new_std = np.std(img[mask>0])
        #
        #  Pulling the background color up to the lower end
        #  of the pixel range for the lungs
        #
        old_min = np.min(img)       # background color
        img[img==old_min] = new_mean-1.2*new_std   # resetting backgound color
        img = img-new_mean
        img = img/new_std
        #make image bounding box  (min row, min col, max row, max col)
        labels = measure.label(mask)
        regions = measure.regionprops(labels)
        #
        # Finding the global min and max row over all regions
        #
        min_row = 512
        max_row = 0
        min_col = 512
        max_col = 0
        for prop in regions:
            B = prop.bbox
            if min_row > B[0]:
                min_row = B[0]
            if min_col > B[1]:
                min_col = B[1]
            if max_row < B[2]:
                max_row = B[2]
            if max_col < B[3]:
                max_col = B[3]
        width = max_col-min_col
        height = max_row - min_row
        if width > height:
            max_row=min_row+width
        else:
            max_col = min_col+height
        # 
        # cropping the image down to the bounding box for all regions
        # (there's probably an skimage command that can do this in one line)
        # 
        img = img[min_row:max_row,min_col:max_col]
        mask =  mask[min_row:max_row,min_col:max_col]
        if max_row-min_row <5 or max_col-min_col<5:  # skipping all images with no god regions
            pass
        else:
            # moving range to -1 to 1 to accomodate the resize function
            mean = np.mean(img)
            img = img - mean
            min = np.min(img)
            max = np.max(img)
            img = img/(max-min)
            new_img = resize(img,[512,512])
            new_node_mask = resize(node_mask[min_row:max_row,min_col:max_col],[512,512])
            out_images.append(new_img)
            out_nodemasks.append(new_node_mask)


working on file  /home/jtu9/lung/output/lungmask_0495_0679.npy
working on file  /home/jtu9/lung/output/lungmask_0724_0346.npy
working on file  /home/jtu9/lung/output/lungmask_0794_0896.npy
working on file  /home/jtu9/lung/output/lungmask_0166_0727.npy
working on file  /home/jtu9/lung/output/lungmask_0251_0097.npy
working on file  /home/jtu9/lung/output/lungmask_0852_0237.npy
working on file  /home/jtu9/lung/output/lungmask_0554_0419.npy
working on file  /home/jtu9/lung/output/lungmask_0231_0105.npy
working on file  /home/jtu9/lung/output/lungmask_0185_0740.npy
working on file  /home/jtu9/lung/output/lungmask_0816_0223.npy
working on file  /home/jtu9/lung/output/lungmask_0573_1126.npy
working on file  /home/jtu9/lung/output/lungmask_0325_0884.npy
working on file  /home/jtu9/lung/output/lungmask_0503_1096.npy
working on file  /home/jtu9/lung/output/lungmask_0483_0310.npy
working on file  /home/jtu9/lung/output/lungmask_0469_0120.npy
working on file  /home/jtu9/lung/output/lungmask_0658_1

/home/jtu9/lung/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jtu9/lung/lib/python3.5/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jtu9/lung/lib/python3.5/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jtu9/lung/lib/python3.5/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jtu9/lung/lib/python3.5/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


working on file  /home/jtu9/lung/output/lungmask_0426_0465.npy
working on file  /home/jtu9/lung/output/lungmask_0682_0667.npy
working on file  /home/jtu9/lung/output/lungmask_0444_0024.npy
working on file  /home/jtu9/lung/output/lungmask_0044_0543.npy
working on file  /home/jtu9/lung/output/lungmask_0594_0488.npy
working on file  /home/jtu9/lung/output/lungmask_0418_0206.npy
working on file  /home/jtu9/lung/output/lungmask_0286_0306.npy
working on file  /home/jtu9/lung/output/lungmask_0088_0786.npy
working on file  /home/jtu9/lung/output/lungmask_0737_0860.npy
working on file  /home/jtu9/lung/output/lungmask_0061_0746.npy
working on file  /home/jtu9/lung/output/lungmask_0887_0468.npy
working on file  /home/jtu9/lung/output/lungmask_0010_0376.npy
working on file  /home/jtu9/lung/output/lungmask_0613_0995.npy
working on file  /home/jtu9/lung/output/lungmask_0839_0534.npy
working on file  /home/jtu9/lung/output/lungmask_0877_0553.npy
working on file  /home/jtu9/lung/output/lungmask_0584_0

working on file  /home/jtu9/lung/output/lungmask_0176_0555.npy
working on file  /home/jtu9/lung/output/lungmask_0471_0800.npy
working on file  /home/jtu9/lung/output/lungmask_0256_0955.npy
working on file  /home/jtu9/lung/output/lungmask_0134_1163.npy
working on file  /home/jtu9/lung/output/lungmask_0692_0032.npy
working on file  /home/jtu9/lung/output/lungmask_0802_0475.npy
working on file  /home/jtu9/lung/output/lungmask_0647_0688.npy
working on file  /home/jtu9/lung/output/lungmask_0157_0291.npy
working on file  /home/jtu9/lung/output/lungmask_0329_0058.npy
working on file  /home/jtu9/lung/output/lungmask_0178_1056.npy
working on file  /home/jtu9/lung/output/lungmask_0812_0600.npy
working on file  /home/jtu9/lung/output/lungmask_0106_1026.npy
working on file  /home/jtu9/lung/output/lungmask_0338_0521.npy
working on file  /home/jtu9/lung/output/lungmask_0503_1097.npy
working on file  /home/jtu9/lung/output/lungmask_0044_0542.npy
working on file  /home/jtu9/lung/output/lungmask_0148_0

working on file  /home/jtu9/lung/output/lungmask_0426_0462.npy
working on file  /home/jtu9/lung/output/lungmask_0789_0923.npy
working on file  /home/jtu9/lung/output/lungmask_0187_0792.npy
working on file  /home/jtu9/lung/output/lungmask_0461_0187.npy
working on file  /home/jtu9/lung/output/lungmask_0189_0180.npy
working on file  /home/jtu9/lung/output/lungmask_0114_0550.npy
working on file  /home/jtu9/lung/output/lungmask_0479_0334.npy
working on file  /home/jtu9/lung/output/lungmask_0426_0464.npy
working on file  /home/jtu9/lung/output/lungmask_0322_0036.npy
working on file  /home/jtu9/lung/output/lungmask_0291_0437.npy
working on file  /home/jtu9/lung/output/lungmask_0459_0968.npy
working on file  /home/jtu9/lung/output/lungmask_0322_0042.npy
working on file  /home/jtu9/lung/output/lungmask_0215_0638.npy
working on file  /home/jtu9/lung/output/lungmask_0763_0268.npy
working on file  /home/jtu9/lung/output/lungmask_0662_0528.npy
working on file  /home/jtu9/lung/output/lungmask_0129_0

...

working on file  /home/jtu9/lung/output/lungmask_0002_1016.npy
working on file  /home/jtu9/lung/output/lungmask_0535_0448.npy
working on file  /home/jtu9/lung/output/lungmask_0884_0824.npy
working on file  /home/jtu9/lung/output/lungmask_0307_1168.npy
working on file  /home/jtu9/lung/output/lungmask_0565_1099.npy
working on file  /home/jtu9/lung/output/lungmask_0049_0051.npy
working on file  /home/jtu9/lung/output/lungmask_0242_0888.npy
working on file  /home/jtu9/lung/output/lungmask_0394_0574.npy
working on file  /home/jtu9/lung/output/lungmask_0409_0848.npy
working on file  /home/jtu9/lung/output/lungmask_0783_1179.npy
working on file  /home/jtu9/lung/output/lungmask_0440_1057.npy
working on file  /home/jtu9/lung/output/lungmask_0581_0454.npy
working on file  /home/jtu9/lung/output/lungmask_0785_0025.npy
working on file  /home/jtu9/lung/output/lungmask_0489_0913.npy
working on file  /home/jtu9/lung/output/lungmask_0299_0427.npy
working on file  /home/jtu9/lung/output/lungmask_0489_0

working on file  /home/jtu9/lung/output/lungmask_0497_0069.npy
working on file  /home/jtu9/lung/output/lungmask_0805_0777.npy
working on file  /home/jtu9/lung/output/lungmask_0185_0742.npy
working on file  /home/jtu9/lung/output/lungmask_0082_0361.npy
working on file  /home/jtu9/lung/output/lungmask_0261_0143.npy
working on file  /home/jtu9/lung/output/lungmask_0451_0609.npy
working on file  /home/jtu9/lung/output/lungmask_0102_0717.npy
working on file  /home/jtu9/lung/output/lungmask_0310_1106.npy
working on file  /home/jtu9/lung/output/lungmask_0450_0421.npy
working on file  /home/jtu9/lung/output/lungmask_0709_0616.npy
working on file  /home/jtu9/lung/output/lungmask_0478_0686.npy
working on file  /home/jtu9/lung/output/lungmask_0455_1093.npy
working on file  /home/jtu9/lung/output/lungmask_0672_0507.npy
working on file  /home/jtu9/lung/output/lungmask_0598_0710.npy
working on file  /home/jtu9/lung/output/lungmask_0010_0377.npy
working on file  /home/jtu9/lung/output/lungmask_0436_0

working on file  /home/jtu9/lung/output/lungmask_0409_0850.npy
working on file  /home/jtu9/lung/output/lungmask_0460_0122.npy
working on file  /home/jtu9/lung/output/lungmask_0293_0633.npy
working on file  /home/jtu9/lung/output/lungmask_0018_1119.npy
working on file  /home/jtu9/lung/output/lungmask_0755_0245.npy
working on file  /home/jtu9/lung/output/lungmask_0157_0292.npy
working on file  /home/jtu9/lung/output/lungmask_0737_0861.npy
working on file  /home/jtu9/lung/output/lungmask_0357_1009.npy
working on file  /home/jtu9/lung/output/lungmask_0606_0628.npy
working on file  /home/jtu9/lung/output/lungmask_0323_1113.npy
working on file  /home/jtu9/lung/output/lungmask_0347_0959.npy
working on file  /home/jtu9/lung/output/lungmask_0717_0565.npy
working on file  /home/jtu9/lung/output/lungmask_0479_0332.npy
working on file  /home/jtu9/lung/output/lungmask_0301_0201.npy
working on file  /home/jtu9/lung/output/lungmask_0414_0859.npy
working on file  /home/jtu9/lung/output/lungmask_0299_0

working on file  /home/jtu9/lung/output/lungmask_0208_0499.npy
working on file  /home/jtu9/lung/output/lungmask_0185_0739.npy
working on file  /home/jtu9/lung/output/lungmask_0790_0067.npy
working on file  /home/jtu9/lung/output/lungmask_0694_0272.npy
working on file  /home/jtu9/lung/output/lungmask_0097_0467.npy
working on file  /home/jtu9/lung/output/lungmask_0567_1023.npy
working on file  /home/jtu9/lung/output/lungmask_0721_0195.npy
working on file  /home/jtu9/lung/output/lungmask_0428_0135.npy
working on file  /home/jtu9/lung/output/lungmask_0174_0737.npy
working on file  /home/jtu9/lung/output/lungmask_0288_0089.npy
working on file  /home/jtu9/lung/output/lungmask_0394_0572.npy
working on file  /home/jtu9/lung/output/lungmask_0368_0669.npy
working on file  /home/jtu9/lung/output/lungmask_0082_0360.npy
working on file  /home/jtu9/lung/output/lungmask_0224_1104.npy
working on file  /home/jtu9/lung/output/lungmask_0692_0033.npy
working on file  /home/jtu9/lung/output/lungmask_0079_0

working on file  /home/jtu9/lung/output/lungmask_0530_0486.npy
working on file  /home/jtu9/lung/output/lungmask_0038_0068.npy
working on file  /home/jtu9/lung/output/lungmask_0026_0869.npy
working on file  /home/jtu9/lung/output/lungmask_0670_0846.npy
working on file  /home/jtu9/lung/output/lungmask_0258_0106.npy
working on file  /home/jtu9/lung/output/lungmask_0329_0057.npy
working on file  /home/jtu9/lung/output/lungmask_0198_1019.npy
working on file  /home/jtu9/lung/output/lungmask_0734_0586.npy
working on file  /home/jtu9/lung/output/lungmask_0355_0796.npy
working on file  /home/jtu9/lung/output/lungmask_0394_0569.npy
working on file  /home/jtu9/lung/output/lungmask_0243_0172.npy
working on file  /home/jtu9/lung/output/lungmask_0394_0570.npy
working on file  /home/jtu9/lung/output/lungmask_0012_0524.npy
working on file  /home/jtu9/lung/output/lungmask_0720_0219.npy
working on file  /home/jtu9/lung/output/lungmask_0694_0276.npy
working on file  /home/jtu9/lung/output/lungmask_0302_0

In [6]:

num_images = len(out_images)
#
#  Writing out images and masks as 1 channel arrays for input into network
#
final_images = np.ndarray([num_images,1,512,512],dtype=np.float32)
final_masks = np.ndarray([num_images,1,512,512],dtype=np.float32)
for i in range(num_images):
    final_images[i,0] =  out_images[i]
    final_masks[i,0] = out_nodemasks[i]

rand_i = np.random.choice(range(num_images),size=num_images,replace=False)
test_i = int(0.2*num_images)
np.save(working_path+"trainImages.npy",final_images[rand_i[test_i:]])
np.save(working_path+"trainMasks.npy",final_masks[rand_i[test_i:]])
np.save(working_path+"testImages.npy",final_images[rand_i[:test_i]])
np.save(working_path+"testMasks.npy",final_masks[rand_i[:test_i]])